In [ ]:
using PerlaTonettiWaugh, Plots, BenchmarkTools

Set up parameters and find the corresponding stationary solution:

In [ ]:
z_min = 0.0 
z_max = 5.0
M = 1000
z_grid = range(z_min, stop = z_max, length = M) # Since we only care about the grid. 

# Define common objects. 
d_0 = 5
d_T = 2.3701
params = (ρ = 0.02, σ = 4.2508, N = 10, θ = 5.1269, γ = 1.00, κ = 0.013, ζ = 1, η = 0, Theta = 1, χ = 1/(2.1868), υ = 0.0593, μ = 0, δ = 0.053, d = 2.3701) # Baselines per Jesse. 
stationary_sol = stationary_numerical(params, z_grid) # solution at t = T
println("g = $(stationary_sol.g), z_hat = $(stationary_sol.z_hat), Ω = $(stationary_sol.Ω)")

Choose `T` and `Ω_vec0` with its corresponding nodes `Ω_nodes`:

In [ ]:
T = 5.0
Ω(t) = stationary_sol.Ω # population fcn

Ω_nodes = 0:1e-1:T
Ω_vec0 = map(t -> Ω(t), Ω_nodes)

Set up `E(t)` and  nodes to be used for entry residuals:

In [ ]:
settings = (z = z_grid, tstops = 0:1e-3:T, Δ_E = 1e-06)
E(t) = (log(Ω(t+settings.Δ_E)) - log(Ω(t-settings.Δ_E)))/(2*settings.Δ_E) + params.δ

entry_residuals_nodes = 1e-1:1e-1:(T-1e-1)

Return the entry residuals:

In [ ]:
# Solve and compute residuals, now using vectorized Ω
@time solved = entry_residuals(params, stationary_sol, settings, T, Ω_vec0, Ω_nodes, entry_residuals_nodes);

Draw the plots!

In [ ]:
plot(entry_residuals_nodes, solved.entry_residuals, label = "entry_residual", lw = 3)

Now using the interpolation:

In [ ]:
plot(entry_residuals_nodes, map(solved.entry_residuals_interpolation, entry_residuals_nodes), label = "entry_residual", lw = 3)